In [16]:
from dataclasses import dataclass, field
import math

# Undirected Graph using Adjacency List

In [32]:
@dataclass(eq=True, frozen=True)
class Node:
    id: int
    attributes: dict = field(default_factory=dict, compare=False, hash=False)

@dataclass
class Graph:
    nodes: list[Node] = field(default_factory=list)
    edges: list[tuple] = field(default_factory=list)
    G: dict = field(default_factory=dict)
    degree: dict = field(default_factory=dict)

    def number_of_nodes(self):
        return len(self.G)

    def get_node(self, node_id: int) -> Node:
        for n in self.nodes:
            if n.id == node_id:
                return n
        new_node = Node(node_id)
        self.nodes.append(new_node)
        self.G[new_node] = []
        return new_node

    def add_node(self, node_id: int, attributes: dict = {}) -> None:
        if any(n.id == node_id for n in self.nodes):
            raise IndexError("Vertice already exists")
        node = Node(node_id, attributes)
        self.nodes.append(node)
        self.G[node] = []

    def add_edge(self, value: tuple) -> None:
        if len(value) != 2:
            raise AssertionError("The edge must be a 2-tuple")

        v1 = self.get_node(value[0])
        v2 = self.get_node(value[1])

        self.G[v1].append(v2)
        self.G[v2].append(v1)
        self.edges.append(value)
        self.degree[v1] = len(self.G[v1])
        self.degree[v2] = len(self.G[v2])

    def dfs(self, start_id: int) -> tuple:
        start = self.get_node(start_id) 
        cpre = 0
        pre = {}
        dfs_tree = []

        def run(u, p=None):
            nonlocal cpre
            cpre += 1
            pre[u] = cpre

            for w in self.G[u]:
                if w not in pre:
                    dfs_tree.append((u, w))
                    run(w, u)

        run(start)
        return pre, dfs_tree
    
    def is_connected(self) -> bool:
        cpre = 0
        pre = {}
        c = 0

        def run(u, p=None):
            nonlocal cpre
            cpre += 1
            pre[u] = cpre

            for w in self.G[u]:
                if w not in pre:
                    run(w, u)
        
        for node in self.nodes:
            if node not in pre:
                c += 1
                run(node)
        return c <= 1
    
    def connected_components(self) -> tuple:
        pass

    def bridges(self) -> list[tuple] :
        cpre = 0
        pre = {}
        low = {}
        bridges = []

        def run(u, p=None):
            nonlocal cpre
            cpre += 1
            pre[u] = cpre
            low[u] = pre[u]

            for w in self.G[u]:
                if w not in pre:
                    run(w, u)
                    if low[w] == pre[w]:
                        bridges.append((u, w))
                    low[u] = min(low[u], low[w])
                if w != p:
                    low[u] = min(low[u], pre[w])
                    
        for node in self.nodes:
            if node not in pre:
                run(node)
        return bridges
    
    def points(self) -> list :
        cpre = 0
        pre = {}
        low = {}
        pa = {}
        articular_points = []

        def run(u, p=None):
            nonlocal cpre
            cpre += 1
            pre[u] = cpre
            low[u] = cpre
            pa[u] = 0

            for w in self.G[u]:
                if w not in pre:
                    run(w, u)
                    if pre[u] <= low[w]:
                        pa[u] += 1
                    low[u] = min(low[u], low[w])
                if w != p:
                    low[u] = min(low[u], pre[w])
                    
        for node in self.nodes:
            if node not in pre:
                run(node)
        
        for i, node in enumerate(self.nodes):
            if (i == 0) and pa[node] > 1:
                articular_points.append(node)
            elif (i != 0) and pa[node] > 0:
                articular_points.append(node)
                
        return articular_points
        

In [4]:
g = Graph()
edges = [
    (1, 2), (1, 3), (1, 4), (1, 6),
    (2, 7), (2, 8), (2, 10),
    (3, 5),
    (4, 5),
    (6, 10),
    (7, 9),
    (8, 9)
]

for edge in edges:
    g.add_edge(edge)

pre, dfs_tree = g.dfs(2)
print("Ordem de visita (pre):", [n.id for n, t in pre.items()])
print("DFS Tree:", [(u.id, v.id) for u, v in dfs_tree])

Ordem de visita (pre): [2, 1, 3, 5, 4, 6, 10, 7, 9, 8]
DFS Tree: [(2, 1), (1, 3), (3, 5), (5, 4), (1, 6), (6, 10), (2, 7), (7, 9), (9, 8)]


In [8]:
g.is_connected()

True

In [15]:
g.degree[g.get_node(1)]

4

In [9]:
gg = Graph()
gg.add_node(1)
gg.add_node(2)
gg.add_node(3)
gg.add_node(4)
gg.add_node(5)

gg.add_edge((1,2))
gg.add_edge((2,3))
gg.add_edge((3,4))


pre, dfs_tree = gg.dfs(3)
print("Ordem de visita (pre):", [n.id for n, t in pre.items()])
print("DFS Tree:", [(u.id, v.id) for u, v in dfs_tree])

Ordem de visita (pre): [3, 2, 1, 4]
DFS Tree: [(3, 2), (2, 1), (3, 4)]


In [10]:
gg.edges

[(1, 2), (2, 3), (3, 4)]

In [11]:
gg.nodes

[Node(id=1, attributes={}),
 Node(id=2, attributes={}),
 Node(id=3, attributes={}),
 Node(id=4, attributes={}),
 Node(id=5, attributes={})]

In [12]:
gg.is_connected()

False

In [23]:
ggg = Graph()
edges = [
    (1, 2), (1, 3),
    (2, 4), (2, 5),
    (3, 5),
    (4, 5),
    (5, 6),
    (6, 8), (6, 7),
    (8, 7)
]

for edge in edges:
    ggg.add_edge(edge)

ggg.bridges()

[(Node(id=5, attributes={}), Node(id=6, attributes={}))]

In [24]:
ggg.points()

[Node(id=5, attributes={}), Node(id=6, attributes={})]

In [33]:
t = Graph()
t.add_edge((1,2))
t.add_edge( (2,3))
t.points()

[Node(id=2, attributes={})]